# Zadanie 1

In [ ]:
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt

# Ustawienie rozmarów wyświetlanych obrazów
plt.rcParams["figure.figsize"] = (18, 10)

image = cv.imread("images/example.jpg")
image = cv.cvtColor(image, cv.COLOR_BGR2RGB)

kernel = np.array([
    [-1, -1, -1],
    [-1, 8, -1],
    [-1, -1, -1],
])
filtered_image = cv.filter2D(image, -1, kernel=kernel)

plt.imshow(filtered_image)

# Zadanie 2

In [ ]:
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt

plt.rcParams["figure.figsize"] = (18, 10)

image = cv.imread("images/example.png")
image = cv.cvtColor(image, cv.COLOR_BGR2RGB)

height, width, _ = image.shape

image = (image / 255.0).astype(np.float32)

kernel = np.array([
    [0.393, 0.769, 0.189],
    [0.349, 0.689, 0.168],
    [0.272, 0.534, 0.131],
])

# Zapisanie wartosci RGB w macierzy (3, height*width)
# -1 - oblicz automatycznie, nie zmieniajac liczby elementow macierzy (= height*width)
rgb_matrix = image.reshape(-1, 3).T

# Mnozenie macierzy wg wzoru: kernel (3 x 3) * RGB_matrix (3 x height*width) = new_RGB_matrix (3 x height*width)
new_rgb_matrix = np.dot(kernel, rgb_matrix)

# Zmiana ksztaltu macierzy wynikowej z (3 x height*width) na (height x width x 3)
filtered_image = new_rgb_matrix.T.reshape(height, width, 3)

# Przyciecie wartosci do dozwolonego przedzialu [0.0; 1.0]
filtered_image = np.clip(filtered_image, 0, 1)

# Konwersja RGB na wartosci z przedzialu [0; 255] i format uint8
filtered_image = (filtered_image * 255).astype(np.uint8)

plt.imshow(filtered_image)

# Zadanie 3

In [ ]:
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt

plt.rcParams["figure.figsize"] = (18, 10)

image = cv.imread("images/example.png")
image = cv.cvtColor(image, cv.COLOR_BGR2RGB)

height, width, _ = image.shape

kernel = np.array([
    [ 0.229,  0.587,  0.114],   # Y
    [ 0.500, -0.418, -0.082],   # Cr
    [-0.168, -0.331,  0.500]    # Cb
])

# Macierz do dodawania, o rozmiarze (3 x height*width)
bias = np.tile(np.array([[0], [128], [128]]), (1, width * height))

# Macierz wartosci RGB o wymiarach (3 x height*width)
# Konwersja wartosci na format float przed mnozeniem
rgb_matrix = image.reshape(-1, 3).T.astype(np.float32)

# Dodawanie i mnozenie macierzy: bias (3 x height*width) + kernel (3 × 3) * RGB_matrix (3 × height*width)
# Macierz wynikowa: (3 x width*height)
ycrcb_matrix = bias + np.dot(kernel, rgb_matrix)

# Przyciecie wartosci do przedzialu [0, 255]
ycrcb_matrix = np.clip(ycrcb_matrix, 0, 255).astype(np.uint8)

# Zmiana ksztaltu macierzy wynikowej (height x  width x 3)
ycrcb_image = ycrcb_matrix.T.reshape(height, width, 3)

# Wyodrebnienie skladowych Y, Cr i Cb
Y_channel, Cr_channel, Cb_channel = cv.split(ycrcb_image)

# Wyswietlenie obrazu oryginalnego i obliczonych skladowych Y, Cr, Cb w odcieniach szarosci
fig, axes = plt.subplots(2, 2, figsize=(30, 20))
axes[0, 0].imshow(image)
axes[0, 0].set_title("Oryginalny obraz", fontsize=20)
axes[0, 1].imshow(Y_channel, cmap="gray")
axes[0, 1].set_title("Skladowa Y (luminancja)", fontsize=20)
axes[1, 0].imshow(Cb_channel, cmap="gray")
axes[1, 0].set_title("Skladowa Cb (chrominancja)", fontsize=20)
axes[1, 1].imshow(Cr_channel, cmap="gray")
axes[1, 1].set_title("Skladowa Cr (chrominancja)", fontsize=20)

for ax in axes.flat:
    ax.axis("off")

plt.show()

# Zadanie 4

In [ ]:
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt

plt.rcParams["figure.figsize"] = (18, 10)

image = cv.imread("images/example.png")
image = cv.cvtColor(image, cv.COLOR_BGR2RGB)


# Krok 1. - Konwersja obrazu z modelu RGB na YCbCr (z zad. 3)

height, width, _ = image.shape

kernel = np.array([
    [ 0.229,  0.587,  0.114],   # Y
    [ 0.500, -0.418, -0.082],   # Cr
    [-0.168, -0.331,  0.500]    # Cb
])

bias = np.tile(np.array([[0], [128], [128]]), (1, width * height))

rgb_matrix = image.reshape(-1, 3).T.astype(np.float32)

ycrcb_matrix = bias + np.dot(kernel, rgb_matrix)

ycrcb_matrix = np.clip(ycrcb_matrix, 0, 255).astype(np.uint8)

ycrcb_image = ycrcb_matrix.T.reshape(height, width, 3)

Y, Cr, Cb = cv.split(ycrcb_image)


# Krok 2. - Operacja downsamplingu na kanałach Cb i Cr

def downsample(channel):
    return channel[::2, ::2] # co drugi rzad i co druga kolumna = co 4 piksel

Cr_downsampled = downsample(Cr)
Cb_downsampled = downsample(Cb)


# Krok 3. - Operacja upsamplingu na kanalach Cb i Cr

def upsample(channel, shape):
    return cv.resize(channel, (shape[1], shape[0]), interpolation=cv.INTER_NEAREST)

Cr_upsampled = upsample(Cr_downsampled, Cr.shape)
Cb_upsampled = upsample(Cb_downsampled, Cb.shape)


# Krok 4. - Zlozenie obrazu z otrzymanych wartosci + ponowna konwersja na RGB

image_ycrcb_reconstructed = cv.merge([Y, Cr_upsampled, Cb_upsampled])
image_rgb_reconstructed = cv.cvtColor(image_ycrcb_reconstructed, cv.COLOR_YCrCb2RGB)


# Krok 5. - Wyswietlenie wynikow

fig, axes = plt.subplots(2, 2, figsize=(30, 20))

axes[0, 0].imshow(image)
axes[0, 0].set_title("Oryginalny obraz RGB", fontsize=24)

axes[0, 1].imshow(Y, cmap="gray")
axes[0, 1].set_title("Kanal Y (luminancja)", fontsize=24)

axes[1, 0].imshow(Cr, cmap="gray")
axes[1, 0].set_title("Oryginalny kanal Cr (chrominancja)", fontsize=24)

axes[1, 1].imshow(Cb, cmap="gray")
axes[1, 1].set_title("Oryginalny kanal Cb (chrominancja)", fontsize=24)

for ax in axes.flat:
    ax.axis("off")

plt.tight_layout()
plt.show()


fig, axes = plt.subplots(2, 2, figsize=(30, 20))

axes[0, 0].imshow(Cr_upsampled, cmap="gray")
axes[0, 0].set_title("Kanal Cr po downsamplingu i upsamplingu", fontsize=24)

axes[0, 1].imshow(Cb_upsampled, cmap="gray")
axes[0, 1].set_title("Kanal Cb po downsamplingu i upsamplingu", fontsize=24)

axes[1, 0].imshow(image)
axes[1, 0].set_title("Oryginalny obraz RGB", fontsize=24)

axes[1, 1].imshow(image_rgb_reconstructed)
axes[1, 1].set_title("Zrekonstruowany obraz RGB", fontsize=24)

for ax in axes.flat:
    ax.axis("off")

plt.tight_layout()
plt.show()

# Zadanie 5

In [ ]:
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt

plt.rcParams["figure.figsize"] = (18, 10)

image = cv.imread("images/example.png")
image = cv.cvtColor(image, cv.COLOR_BGR2RGB)


# Krok 1. - Konwersja obrazu z modelu RGB na YCbCr (z zad. 3)

height, width, _ = image.shape

kernel = np.array([
    [ 0.229,  0.587,  0.114],   # Y
    [ 0.500, -0.418, -0.082],   # Cr
    [-0.168, -0.331,  0.500]    # Cb
])

bias = np.tile(np.array([[0], [128], [128]]), (1, width * height))

rgb_matrix = image.reshape(-1, 3).T.astype(np.float32)

ycrcb_matrix = bias + np.dot(kernel, rgb_matrix)

ycrcb_matrix = np.clip(ycrcb_matrix, 0, 255).astype(np.uint8)

ycrcb_image = ycrcb_matrix.T.reshape(height, width, 3)

Y, Cr, Cb = cv.split(ycrcb_image)


# Krok 2. - Operacja downsamplingu na kanałach Cb i Cr

def downsample(channel):
    return channel[::2, ::2] # Take every second row and column

Cr_downsampled = downsample(Cr)
Cb_downsampled = downsample(Cb)


# Krok 3. - Operacja upsamplingu na kanalach Cb i Cr

def upsample(channel, shape):
    return cv.resize(channel, (shape[1], shape[0]), interpolation=cv.INTER_NEAREST)

Cr_upsampled = upsample(Cr_downsampled, Cr.shape)
Cb_upsampled = upsample(Cb_downsampled, Cb.shape)


# Krok 4. - Zlozenie obrazu z otrzymanych wartosci + ponowna konwersja na RGB

image_ycrcb_reconstructed = cv.merge([Y, Cr_upsampled, Cb_upsampled])
image_rgb_reconstructed = cv.cvtColor(image_ycrcb_reconstructed, cv.COLOR_YCrCb2RGB)


# Krok 5. - Wyznaczenie wartosci bledu sredniokwadratowego

def calculate_mse(original, reconstructed):
    original = original.astype(np.float32)
    reconstructed = reconstructed.astype(np.float32)
    
    diff = original - reconstructed
    mse = np.mean(np.square(diff))
    return mse

mse_value = calculate_mse(image, image_rgb_reconstructed)
print(f"Calculated MSE value: \033[1m{mse_value:.2f}\033[0m")